In [6]:
install_packages = True

In [43]:
%pip install huggingface-hub

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.document_loaders import DirectoryLoader, CSVLoader, UnstructuredWordDocumentLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from sentence_transformers import SentenceTransformer, util
#from htmlTemplate import css, bot_template, user_template
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_history_aware_retriever
from langchain_huggingface import HuggingFaceEmbeddings

/Users/gonzaloguaimas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gonzaloguaimas/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
file_directory="/Users/gonzaloguaimas/Documents/code/AI/motoplan-rag/data"
embedding_model='sentence-transformers/all-MiniLM-L6-v2'
llm_model ="llama3.2"

In [4]:
def prepare_and_split_docs(directory):
    # Load the documents
    loaders = [
        DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
        DirectoryLoader(directory, glob="**/*.docx",show_progress=True),
        DirectoryLoader(directory, glob="**/*.csv",loader_cls=CSVLoader)
    ]


    documents=[]
    for loader in loaders:
        data =loader.load()
        documents.extend(data)

    # Initialize a text splitter
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,  # Use the smaller chunk size here to avoid repeating splitting logic
        chunk_overlap=256,
        disallowed_special=(),
        separators=["\n\n", "\n", " "]
    )

    # Split the documents and keep metadata
    split_docs = splitter.split_documents(documents)

    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs

In [6]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
def ingest_into_vectordb(split_docs):
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    print("Documents are inserted into FAISS vectorstore")
    return db

In [37]:
def get_conversation_chain(retriever):
    llm = Ollama(model=llm_model)
    contextualize_q_system_prompt = (
        "Teniendo la última conversación en cuenta, contextualizá la pregunta del usuario."
        "Si el usuario necesita una respuesta específica, dá la respuesta en base a los documentos con lo que te entrené. Si te dice: Buen día, ¿cómo estás?, respondé: Buen día, estoy bien, gracias. ¿En qué puedo ayudarte?, es decir respuestas cortas y concisas."
        "No hagás muchas preguntas, siempre tratá de recomendar que valla a una oficina en salta o tucumán o que diga en qué horario se lo puede llamar"
    )


    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    ### Answer question ###
    system_prompt = (
        "Sos un asesor comercial de la empresa Motoplan18, tenés que responder de manera tranquila y humana a cada respuesta que te hagan. "
        "Tratá de usar el lenguaje argentino pero no vulgar. Respondé en 1 sóla oración con no más de 15 palabras. "
        "Contestá de manera natural, no te sobreextiendas ni hagas muchas preguntas. No hables en neutro, hablá como argentino. "
        "{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


    ### Statefully manage chat history ###
    store = {}


    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    print("Conversational chain created")
    return conversational_rag_chain


In [8]:
def calculate_similarity_score(answer: str, context_docs: list) -> float:
       
    context_docs = [doc.page_content for doc in context_docs]
    
    # Encode the answer and context documents
    answer_embedding = embeddings.embed_query(answer)
    context_embeddings = embeddings.embed_documents(context_docs)
    
    # Calculate cosine similarities
    similarities = util.pytorch_cos_sim(answer_embedding, context_embeddings)
    
    # Return the maximum similarity score from the context documents
    max_score = similarities.max().item() 
    return max_score

In [40]:
split_docs=prepare_and_split_docs(file_directory)
vector_db= ingest_into_vectordb(split_docs)
retriever =vector_db.as_retriever()
conversational_rag_chain=get_conversation_chain(retriever)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
100%|██████████| 2/2 [00:00<00:00, 37.30it/s]
0it [00:00, ?it/s]

Documents are split into 9 passages
Documents are inserted into FAISS vectorstore
Conversational chain created


In [43]:
user_question="No tengo recibo de sueldo y quiero una yamaha"

In [44]:
qa1=conversational_rag_chain.invoke(
    {"input": user_question},
    config={
        "configurable": {"session_id": "abc123"}
    }
)
print(qa1["answer"])

Amigo, para sacar una Yamaha en cuotas fijas, necesitás saber si tienes DNI sin problemas con la firma, o tenemos un plan nacional 110 para ti. ¿Qué pasa?


In [35]:
print("Conversation Chain")
print(qa1)

Conversation Chain
{'input': 'gracias', 'chat_history': [HumanMessage(content='Hola como puedo hacer para tener una moto soy de salta capital', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Hola! Puedes contactarnos en nuestra sucursal de Salta Capital, en Pelegrinni 664, o en Joaquín V. González en Hipolito Yrigoyen 123, y hablaremos de los planes de financiamiento que podemos ofrecerte.', additional_kwargs={}, response_metadata={}), HumanMessage(content='a la tarde hasata quihora estan?', additional_kwargs={}, response_metadata={}), AIMessage(content='Estamos abiertos del martes al viernes de 10am a 7pm, así que si necesitas hablar con nosotros a la tarde, puedes venir a nuestra sucursal en Pelegrinni 664.', additional_kwargs={}, response_metadata={}), HumanMessage(content='información para la moto', additional_kwargs={}, response_metadata={}), AIMessage(content='¡Excelente! Entonces necesitamos saber si tiene DNI sin la firma afectada. ¿Puede mostrarnos su documen

In [36]:
answer = qa1["answer"]
context_docs = qa1["context"]
similarity_score = calculate_similarity_score(answer, context_docs)

print("Context Similarity Score:", round(similarity_score,2))

Context Similarity Score: 0.63
